In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


<a id='top'></a>
## Index

[Bot](#bot)



## Imports

In [6]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from __future__ import division

## Import Datas for dummy Baselines

In [3]:
datas = pd.read_csv('/home/louis/kaggle/allen/sample_submission.csv')

In [4]:
A = ['A' for i in datas.index]
B = ['B' for i in datas.index]
C = ['C' for i in datas.index]
D = ['D' for i in datas.index]

In [5]:
datas.correctAnswer=A
datas.to_csv('/home/louis/kaggle/allen/submission_all_A.csv',index=False)
datas.correctAnswer=B
datas.to_csv('/home/louis/kaggle/allen/submission_all_B.csv',index=False)
datas.correctAnswer=C
datas.to_csv('/home/louis/kaggle/allen/submission_all_C.csv',index=False)
datas.correctAnswer=D
datas.to_csv('/home/louis/kaggle/allen/submission_all_D.csv',index=False)


## Import Datas & explo

In [7]:
train = pd.read_csv('/home/louis/kaggle/allen/training_set.tsv',sep='\t')
valid = pd.read_csv('/home/louis/kaggle/allen/validation_set.tsv',sep='\t')
answers=['answerA','answerB','answerC','answerD']

In [7]:
train.head()

,id,question,correctAnswer,answerA,answerB,answerC,answerD
0,100001,"When athletes begin to exercise, their heart r...",C,at the tissue level,at the organ level,at the system level,at the cellular level
1,100002,Which example describes a learned behavior in ...,C,smelling the air for odors,barking when disturbed,sitting on command,digging in soil
2,100003,"When two nuclei are combined into one nucleus,...",D,conversion,reaction,fission,fusion
3,100004,Which is a distinction between an epidemic and...,B,the symptoms of the disease,the geographical area affected,the species of organisms infected,the season in which the disease spreads
4,100005,In which way is the orbit of a comet different...,B,The orbit of Earth is less circular than the o...,The orbit of a comet is more elliptical than t...,The orbital period of Earth is much longer tha...,The orbital period of a comet is more predicta...


In [8]:
answers=['answerA','answerB','answerC','answerD']
def print_q_a(df,index):
    print df.ix[index].question
    print df.ix[index][answers]

In [9]:
print_q_a(train,2)

When two nuclei are combined into one nucleus, there is a slight change in mass and the release of a large amount of energy. What is this process called?
answerA    conversion
answerB      reaction
answerC       fission
answerD        fusion
Name: 2, dtype: object


## using Word2Vec on training datas

In [3]:
import nltk
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import RegexpTokenizer

### preparing the sentences and playing with Word2Vec

In [8]:
tokenizer=RegexpTokenizer(r'\w+')
sentences = [tokenizer.tokenize(train.ix[i].question.lower()+' '+train.ix[i]['answer'+train.ix[i]['correctAnswer']].lower())for i in train.index]

In [9]:
sentences

[['when',
  'athletes',
  'begin',
  'to',
  'exercise',
  'their',
  'heart',
  'rates',
  'and',
  'respiration',
  'rates',
  'increase',
  'at',
  'what',
  'level',
  'of',
  'organization',
  'does',
  'the',
  'human',
  'body',
  'coordinate',
  'these',
  'functions',
  'at',
  'the',
  'system',
  'level'],
 ['which',
  'example',
  'describes',
  'a',
  'learned',
  'behavior',
  'in',
  'a',
  'dog',
  'sitting',
  'on',
  'command'],
 ['when',
  'two',
  'nuclei',
  'are',
  'combined',
  'into',
  'one',
  'nucleus',
  'there',
  'is',
  'a',
  'slight',
  'change',
  'in',
  'mass',
  'and',
  'the',
  'release',
  'of',
  'a',
  'large',
  'amount',
  'of',
  'energy',
  'what',
  'is',
  'this',
  'process',
  'called',
  'fusion'],
 ['which',
  'is',
  'a',
  'distinction',
  'between',
  'an',
  'epidemic',
  'and',
  'a',
  'pandemic',
  'the',
  'geographical',
  'area',
  'affected'],
 ['in',
  'which',
  'way',
  'is',
  'the',
  'orbit',
  'of',
  'a',
  'comet'

In [10]:
model = Word2Vec(sentences)

/home/louis/anaconda/lib/python2.7/site-packages/gensim/models/word2vec.py:665: UserWarning: C extension not loaded for Word2Vec, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension not loaded for Word2Vec, training will be slow. "


In [11]:
model.most_similar('at',topn=5)

[('measure', 0.9661522507667542),
 ('car', 0.9654197692871094),
 ('fact', 0.9631081819534302),
 ('stream', 0.9626766443252563),
 ('planet', 0.9615350365638733)]

In [12]:
model.most_similar('comet',topn=5)

[('gives', 0.8969342708587646),
 ('demonstrates', 0.8965225219726562),
 ('scale', 0.8928073644638062),
 ('asteroid', 0.8912460803985596),
 ('hurricane', 0.8907577395439148)]

In [44]:
model.most_similar(positive=['orbit', 'comet'], negative=['car'], topn=1)

[('composed', 0.9424656629562378)]

In [47]:
model['orbit']+model['comet']

array([ 0.00627436, -0.00863943,  0.00711971,  0.00993563,  0.04586377,
       -0.0026661 , -0.02943576, -0.01944198, -0.00217664,  0.00331423,
       -0.03930401,  0.03454028,  0.01994083,  0.03740608,  0.00478948,
        0.03041484, -0.03881324,  0.02288892, -0.00497102, -0.04476333,
        0.00745278, -0.00252434,  0.02870042,  0.01367306, -0.05401202,
        0.02492355,  0.00239123, -0.02952624,  0.00993743, -0.0227486 ,
        0.0114664 ,  0.01625355, -0.00776305, -0.01319305,  0.01222651,
       -0.02065764,  0.00044113,  0.0059498 , -0.01428063,  0.02998991,
        0.02923672, -0.03860296,  0.0076221 ,  0.00166415, -0.00657617,
        0.00831657,  0.02931798, -0.02291654, -0.00918292, -0.00772634,
       -0.00894566,  0.00630761,  0.02452301,  0.03834424,  0.00987644,
       -0.00848858,  0.02243999, -0.01133064,  0.03042805,  0.0084656 ,
       -0.00842803,  0.00855203, -0.02983861,  0.00592208, -0.0141622 ,
       -0.00205057,  0.01099362, -0.0119169 ,  0.01199801, -0.04

## leave one out cross val on trainset

In [84]:
choices=['A','B','C','D']
model_answers=[]
for ind in train.index:
    sentences = [tokenizer.tokenize(train.ix[i].question.lower()+' '+train.ix[i]['answer'+train.ix[i]['correctAnswer']].lower())for i in train.index if i!=ind]
    model = Word2Vec(sentences)
    question = tokenizer.tokenize(train.ix[ind].question.lower())
    answers = [tokenizer.tokenize(train.ix[ind]['answer'+answer].lower())for answer in choices]
    test = []

    list_question_vects=[]
    for w in question:
        if w in model:
            vect = model[w]
            list_question_vects.append(vect)
    question_vector = sum(list_question_vects,0)

    for a in answers:
        list_ansvect=[]
        for w in a:
            if w in model:
                vect = model[w]
                list_ansvect.append(vect)
        sum_ans_vect = sum(list_ansvect,0)
        test.append(np.dot(question_vector,sum_ans_vect)/np.linalg.norm(sum_ans_vect))
    try:
        print choices[argmax(test)]
        model_answers.append(choices[argmax(test)])
    except:
        # si ça merde on met A... très propre comme méthode
        print "la folie"
        model_answers.append('la folie')
    print ind
            
train['first_try']=model_answers
    

C
0
A
1
A
2
D
3
A
4
C
5
la folie
6
A
7
la folie
8
D
9
D
10
A
11
B
12
A
13
la folie
14
la folie
15
C
16
la folie
17
A
18
A
19
la folie
20
la folie
21
A
22
B
23
C
24
B
25
B
26
B
27
A
28
la folie
29
B
30
la folie
31
D
32
D
33
B
34
D
35
D
36
la folie
37
A
38
D
39
A
40
A
41
A
42
D
43
A
44
D
45
D
46
A
47
A
48
A
49
C
50
la folie
51
D
52
B
53
A
54
A
55
A
56
B
57
C
58
la folie
59
A
60
D
61
B
62
C
63
D
64
la folie
65
A
66
la folie
67
A
68
A
69
D
70
D
71
B
72
D
73
C
74
B
75
la folie
76
D
77
B
78
C
79
C
80
A
81
la folie
82
D
83
la folie
84
C
85
D
86
D
87
la folie
88
A
89
D
90
B
91
la folie
92
A
93
A
94
C
95
la folie
96
C
97
B
98
A
99
la folie
100
B
101
A
102
D
103
A
104
B
105
la folie
106
B
107
A
108
C
109
C
110
D
111
B
112
C
113
C
114
A
115
C
116
C
117
D
118
C
119
A
120
D
121
C
122
la folie
123
C
124
C
125
D
126
C
127
B
128
la folie
129
A
130
C
131
la folie
132
A
133
C
134
D
135
A
136
A
137
D
138
C
139
la folie
140
A
141
A
142
D
143
la folie
144
A
145
B
146
D
147
A
148
B
149
B
150
A
151
B
152
la 

In [ ]:
train['first_try']=model_answers


In [87]:
sum([1 for i in train.index if train.ix[i].first_try ==train.ix[i].correctAnswer])

502

In [98]:
def folie_to_letter(answer,new_letter):
    if answer=='la folie':
        return new_letter
    else:
        return answer
model_answers_A = [folie_to_letter(model_answers[i],'A')for i in range(len(model_answers))]
train['first_try_A']=model_answers_A
print sum([1 for i in train.index if train.ix[i].first_try_A ==train.ix[i].correctAnswer])
print len(train)
print sum([1 for i in train.index if train.ix[i].first_try_A ==train.ix[i].correctAnswer])/len(train)
    

606
2500
0.2424


0.24....ça pu, mais on va tester sur le validation set quand même

### creating submission file for validation set

In [112]:
choices=['A','B','C','D']
model_answers=[]
sentences = [tokenizer.tokenize(train.ix[i].question.lower()+' '+train.ix[i]['answer'+train.ix[i]['correctAnswer']].lower())for i in train.index]
model = Word2Vec(sentences)
for ind in valid.index:

    question = tokenizer.tokenize(valid.ix[ind].question.lower())
    answers = [tokenizer.tokenize(valid.ix[ind]['answer'+answer].lower())for answer in choices]
    test = []

    list_question_vects=[]
    for w in question:
        if w in model:
            vect = model[w]
            list_question_vects.append(vect)
    question_vector = sum(list_question_vects,0)

    for a in answers:
        list_ansvect=[]
        for w in a:
            if w in model:
                vect = model[w]
                list_ansvect.append(vect)
        sum_ans_vect = sum(list_ansvect,0)
        test.append(np.dot(question_vector,sum_ans_vect)/np.linalg.norm(sum_ans_vect))
    try:
        print choices[argmax(test)]
        model_answers.append(choices[argmax(test)])
    except:
        # si ça merde on met A... très propre comme méthode
        print "la folie"
        model_answers.append('A')
    print ind

D
0
la folie
1
D
2
la folie
3
la folie
4
la folie
5
D
6
la folie
7
D
8
la folie
9
la folie
10
C
11
la folie
12
la folie
13
A
14
la folie
15
la folie
16
A
17
la folie
18
la folie
19
D
20
la folie
21
la folie
22
C
23
la folie
24
la folie
25
D
26
C
27
B
28
la folie
29
A
30
B
31
B
32
la folie
33
la folie
34
la folie
35
la folie
36
la folie
37
la folie
38
D
39
la folie
40
C
41
D
42
la folie
43
B
44
la folie
45
la folie
46
A
47
D
48
la folie
49
D
50
la folie
51
la folie
52
A
53
C
54
la folie
55
la folie
56
A
57
la folie
58
la folie
59
A
60
la folie
61
la folie
62
la folie
63
A
64
la folie
65
la folie
66
la folie
67
la folie
68
la folie
69
la folie
70
B
71
la folie
72
la folie
73
la folie
74
C
75
la folie
76
la folie
77
B
78
D
79
D
80
B
81
la folie
82
la folie
83
la folie
84
A
85
la folie
86
la folie
87
A
88
la folie
89
la folie
90
A
91
A
92
la folie
93
la folie
94
la folie
95
la folie
96
C
97
la folie
98
la folie
99
D
100
la folie
101
A
102
la folie
103
B
104
A
105
la folie
106
la folie
107


In [113]:
            
valid['first_try']=model_answers
datas['correctAnswer']=model_answers
datas.to_csv('/home/louis/kaggle/allen/submission_first_try_w2v.csv', index=False)
    

In [102]:
len(valid)

8132

In [104]:
submit = pd.DataFrame(copy(valid),columns=valid.keys())

In [105]:
submit

,id,question,answerA,answerB,answerC,answerD,first_try
0,102501,A meter is industry by developing a process of...,cool foods during shipping,yard,nervous system,birds of prey,D
1,102502,Which part of directly involved in recycling t...,panting,urethra,predators,water,A
2,102503,Which of the seeds grow better in the light or...,muscular system,keep one group in a light place and another in...,skates and rays,nervous tissue,D
3,102504,Which of the hermaphroditic. This growing seas...,coniferous forest,no reproductive organs,there is less water loss through small leaves,perennial,A
4,102505,When marsupials following is not a transport o...,heart.,glucagon.,hatched.,to protect vital organs.,A
5,102506,Why is deficiency disease resulting from foxgl...,potassium.,are woody plants.,to serve as a point of comparison.,cigarettes.,A
6,102507,Which food body system produces by doing all o...,deer -> Grain -> Bear.,ancient pottery.,ridding their bodies of liquid waste often.,circulatory.,D
7,102508,Which pair of terms describes following nitrog...,thymine-adenine,the nasal passageway is larger and can take in...,slime molds,external fertilization and external development,A
8,102509,What is a statement best impact of most other ...,ability to adapt to change,A killed or weakened virus,to collect water and minerals,when the sperm joins the egg,D
9,102510,"The damaging seasonal hurricanes in Florida, a...",the price of oranges has dropped dramatically.,to enable movement.,the common cold.,hallucinogen.,A


In [108]:
datas

,id,correctAnswer
0,102501,D
1,102502,D
2,102503,D
3,102504,D
4,102505,D
5,102506,D
6,102507,D
7,102508,D
8,102509,D
9,102510,D


In [1]:
import wikipedia as wp

In [2]:
wp.search('science')

[u'Science',
 u'Science (disambiguation)',
 u'Science (journal)',
 u'Future Science Fiction and Science Fiction Stories',
 u'Computer science',
 u'Outline of science fiction',
 u'List of science magazines',
 u'List of years in science',
 u'Bachelor of Computer Science',
 u'Outline of health sciences']

In [14]:
wp.page(wp.search('science')[4]).content

u'Computer science is the scientific and practical approach to computation and its applications. It is the systematic study of the feasibility, structure, expression, and mechanization of the methodical procedures (or algorithms) that underlie the acquisition, representation, processing, storage, communication of, and access to information. An alternate, more succinct definition of computer science is the study of automating algorithmic processes that scale. A computer scientist specializes in the theory of computation and the design of computational systems.\nIts fields can be divided into a variety of theoretical and practical disciplines. Some fields, such as computational complexity theory (which explores the fundamental properties of computational and intractable problems), are highly abstract, while fields such as computer graphics emphasize real-world visual applications. Still other fields focus on the challenges in implementing computation. For example, programming language th

<a id='bot'></a>
[Top](#top)